In [ ]:
#Vizualize demand spatial distribution in Amsterdam emoped pilot

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches

import pandas as pd
import re
import xml.etree.ElementTree as ET
import json

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_bbox_origin_destination_layer
from mnms.vehicles.veh_type import Tram, Metro, Bus
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, save_odlayer
from mnms.tools.render import draw_roads, draw_line, draw_odlayer
from mnms.tools.geometry import points_in_polygon, get_bounding_box

In [ ]:
### Parameters

nb_emoped = 600

d_max = 200 # , max distance between hex and node for station

# Files and directories
current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

f = open('params.json')
params = json.load(f)

#amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
#ams_dmd_path = indir + 'test_all_in_highway_7h_9h.csv' # dmd inside ringroad 7-9am
init_emoped_path = indir + 'init_pos_emoped.csv'
fn_stations = indir + 'emoped_stations'+str(nb_emoped)+'.csv'

np.random.seed(68568)

In [ ]:
### Get the MLGraph without TCs 
amsterdam_graph = load_graph(indir+params['fn_network'])
roads = amsterdam_graph.roads

In [ ]:
# Load dmd
db_dmd = pd.read_csv(indir+params['fn_demand'], sep=';')

In [ ]:
db_dmd

In [ ]:
nb_dmd = len(db_dmd)
origins = np.zeros((len(db_dmd),2))
destinations = np.zeros((len(db_dmd),2))
for i, row in db_dmd[:].iterrows():
    origins[i] = [float(o) for o in row['ORIGIN'].split(' ')]
    destinations[i] = [float(d) for d in row['DESTINATION'].split(' ')]

In [ ]:
ho = plt.hexbin(origins[:,0], origins[:,1], gridsize=10, alpha=0.2, cmap='Blues')
cb = plt.colorbar(ho)

In [ ]:
hd = plt.hexbin(destinations[:,0], destinations[:,1], gridsize=10, alpha=0.2, cmap='Blues')
cb = plt.colorbar(hd)

# Free-floating

In [ ]:
# Emoped initial locations
x = np.random.random(nb_emoped)*(origins[:,0].max()-origins[:,0].min()) + origins[:,0].min()
y = np.random.random(nb_emoped)*(origins[:,1].max()-origins[:,1].min()) + origins[:,1].min()
df_emoped = pd.DataFrame({'x':x, 'y':y})

In [ ]:
nodes_pos = np.array([node.position for node in roads.nodes.values()])
nodes_id = list(roads.nodes.keys())

In [ ]:
closest_nodes = []
x_nodes = []
y_nodes = []
for i, row in df_emoped.iterrows():
    x = row['x']
    y = row['y']
    dist_nodes = (x-nodes_pos[:,0])**2 + (y-nodes_pos[:,1])**2
    i_min = np.argmin(dist_nodes)
    closest_nodes.append(nodes_id[i_min])
    x_nodes.append(nodes_pos[i_min,0])
    y_nodes.append(nodes_pos[i_min,1])
df_emoped['closest_node'] = closest_nodes
df_emoped['x_node'] = x_nodes
df_emoped['y_node'] = y_nodes

In [ ]:
df_emoped

In [ ]:
a=df_emoped.value_counts('closest_node')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

plt.plot(origins[:,0], origins[:,1], 'b.', alpha=0.5)
plt.plot(destinations[:,0], destinations[:,1], 'r.', alpha=0.5)
#plt.plot(od_clustered[:,0], od_clustered[:,1], 'gd', alpha=0.5)

plt.plot(df_emoped['x_node'], df_emoped['y_node'],'gd')
plt.plot(df_emoped['x'], df_emoped['y'],'kd')
    
#plt.xlim([624e3, 635e3])
#plt.ylim([5.795e6, 5.8125e6])

# Station locations

## Based on od layer

In [ ]:
f = open(indir+params['fn_odlayer'])
od_layer = json.load(f)

nodes_id = [k for k in roads.nodes.keys() if not ('TRAM' in k or 'BUS' in k or 'METRO' in k)]
nodes_pos = np.array([roads.nodes[k].position for k in nodes_id])
#nodes_pos = np.array([node.position for node in roads.nodes.values()])
#nodes_id = list(roads.nodes.keys())

In [ ]:
nodes_pos

In [ ]:
closest_nodes = []
x_nodes = []
y_nodes = []
for i in od_layer['ORIGINS'].items():
    x = i[1][0]
    y = i[1][1]
    dist_nodes = (x-nodes_pos[:,0])**2 + (y-nodes_pos[:,1])**2
    i_min = np.argmin(dist_nodes)
    closest_nodes.append(nodes_id[i_min])
    x_nodes.append(nodes_pos[i_min,0])
    y_nodes.append(nodes_pos[i_min,1])


df_st_emoped = pd.DataFrame({'closest_node': closest_nodes, 'x_node':x_nodes, 'y_node':y_nodes})

# Remove doppel station
a = df_st_emoped['closest_node'].value_counts()
dbl_id = a[a>1].keys()
for n_id in dbl_id:
    for st_id in df_st_emoped[df_st_emoped.closest_node == n_id][1:].index:
        df_st_emoped.drop(st_id, inplace=True)
df_st_emoped.reset_index(drop=True, inplace=True)

In [ ]:
df_st_emoped

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)
ho = plt.hexbin(origins[:,0], origins[:,1], gridsize=10, alpha=0.2, cmap='Blues')
cb = plt.colorbar(ho)

plt.plot(df_st_emoped['x_node'], df_st_emoped['y_node'],'gd')

In [ ]:
hex_values = ho.get_array()
hex_values/= hex_values.sum()

In [ ]:
hex_centers = ho.get_offsets()

In [ ]:
# Choose nb of emoped depending on demand
st_hex = np.zeros(len(df_st_emoped),dtype=int)
nb_st_per_hex = np.zeros(len(hex_values),dtype=int)

for i_st,(x,y) in enumerate(zip(df_st_emoped.x_node, df_st_emoped.y_node)):
    dist_sq = [(x-x_)**2+(y-y_)**2 for x_,y_ in zip(hex_centers[:,0],hex_centers[:,1])]
    i_hex = np.argmin(dist_sq)
    st_hex[i_st] = i_hex
    nb_st_per_hex[i_hex] += 1
    
nb_emoped_prop = np.zeros(len(df_st_emoped),dtype=int)
for i_st, i_hex in enumerate(st_hex):
    nb_emoped_prop[i_st] = int(nb_emoped * hex_values[i_hex] / nb_st_per_hex[i_hex] + 0.5)



In [ ]:
nb_emoped_prop.sum()

In [ ]:
df_st_emoped['nb_emoped'] = nb_emoped_prop

## Based on Felyx data

In [ ]:
polygon = np.asarray(params['polygon_demand'])

In [ ]:
# Open locations based on Felyx data
df_felyx = pd.read_csv('inputs/processed_felyx_data/virt_station_loc_hex9.csv')

In [ ]:
df_felyx

In [ ]:
coord_xy = [[x,y] for (x,y) in zip(df_felyx.hex_x.values, df_felyx.hex_y.values)]
mask = points_in_polygon(polygon, coord_xy)

In [ ]:
df_sta = df_felyx[mask]
df_sta.reset_index(inplace=True, drop=True)

In [ ]:
# Associate closest node
nodes_id = [k for k in roads.nodes.keys() if not ('TRAM' in k or 'BUS' in k or 'METRO' in k)]
nodes_pos = np.array([roads.nodes[k].position for k in nodes_id])

closest_nodes = []
x_nodes = []
y_nodes = []
for (x,y) in zip(df_sta.hex_x.values, df_sta.hex_y.values):
    dist_nodes = (x-nodes_pos[:,0])**2 + (y-nodes_pos[:,1])**2
    i_min = np.argmin(dist_nodes)
    x_nodes.append(nodes_pos[i_min,0])
    y_nodes.append(nodes_pos[i_min,1])
    if dist_nodes[i_min]>d_max**2:
        closest_nodes.append('to_delete')
    else:
        closest_nodes.append(nodes_id[i_min])


df_sta['closest_node'] = closest_nodes
df_sta['x_node'] = x_nodes
df_sta['y_node'] = y_nodes

df_sta = df_sta[df_sta.closest_node!='to_delete']
df_sta.reset_index(drop=True, inplace=True)

# Remove doppel station
a = df_sta['closest_node'].value_counts()
dbl_id = a[a>1].keys()
for n_id in dbl_id:
    for st_id in df_sta[df_sta.closest_node == n_id][1:].index:
        df_sta.drop(st_id, inplace=True)
df_sta.reset_index(drop=True, inplace=True)



In [ ]:
df_sta['nb_emoped'] = np.zeros(len(df_sta), dtype=int)
for _ in range(nb_emoped):
    i = np.random.choice(df_sta.index)#, p = df_sta['origin_dmd'].values/df_sta['origin_dmd'].sum())
    df_sta['nb_emoped'].iloc[i] += 1

In [ ]:
df_sta

In [ ]:
# VIP station

df_sta_vip = df_sta.sort_values('origin_dmd', ascending=False)[:20]

In [ ]:
# Distance between station

min_dist = 1e6
for i1 in range(len(df_sta)):
    x1 = df_sta.hex_x.values[i1]
    y1 = df_sta.hex_y.values[i1]
    for i2 in range(i1):
        x2 = df_sta.hex_x.values[i2]
        y2 = df_sta.hex_y.values[i2]
        dist_nodes = (x1-x2)**2 + (y1-y2)**2
        if dist_nodes<min_dist:
            min_dist = dist_nodes
print(np.sqrt(min_dist))

In [ ]:
df_sta_vip.closest_node.values

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

#plt.plot(df_sta.hex_x, df_sta.hex_y,'gd',alpha=0.1)
plt.scatter(df_sta.x_node, df_sta.y_node, c=df_sta.nb_emoped)
plt.scatter(df_sta_vip.x_node, df_sta_vip.y_node, c='r', marker='*', alpha=0.5)

In [ ]:
plt.hist(df_sta.nb_emoped)

In [ ]:
## Save file 
df_sta.to_csv(fn_stations)